In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#define a measure strategy
import sys
sys.path.append("..")

In [3]:
#read data
#load dataset with panda    
import pandas as pd

source_file = '../data/env.sel3.sk_sc.csv'
env = pd.read_csv(source_file)


#DEFINE design matrix
Y = env['occupancy']
X = env[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]

X__np = X.to_numpy()
y_np = Y.to_numpy()

In [4]:
#our data
X.describe()

,illuminance,blinds,lamps,rh,co2,temp
count,2.865000e+03,2.865000e+03,2.865000e+03,2.865000e+03,2.865000e+03,2.865000e+03
mean,-5.952190e-17,1.190438e-16,7.440238e-17,8.333067e-16,-4.960159e-16,-4.364940e-16
std,1.000175e+00,1.000175e+00,1.000175e+00,1.000175e+00,1.000175e+00,1.000175e+00
min,-1.016464e+00,-2.153509e+00,-1.219187e+00,-2.867988e+00,-1.222898e+00,-1.774843e+00
25%,-8.577242e-01,5.329308e-01,-1.219187e+00,-4.930550e-01,-8.493488e-01,-8.539926e-01
50%,-6.990397e-02,5.329308e-01,8.248370e-01,2.229178e-01,-3.253382e-01,-1.940488e-01
75%,3.357646e-01,5.329308e-01,8.248370e-01,6.345396e-01,7.570723e-01,7.268020e-01
max,8.560843e+00,5.329308e-01,8.248370e-01,1.996634e+00,3.584325e+00,2.614547e+00


In [5]:
#the label
Y.describe()

count    2865.000000
mean       -0.130192
std         0.991662
min        -1.000000
25%        -1.000000
50%        -1.000000
75%         1.000000
max         1.000000
Name: occupancy, dtype: float64

In [6]:
#shape of ndarray
print(X__np.shape)
print(y_np.shape)

(2865, 6)
(2865,)


In [7]:
#generate obs
from pqk.aux_funcs import generate_my_obs

obs = generate_my_obs(['X','Y','Z'], n_qub=6)
print(obs)

['XIIIII', 'IXIIII', 'IIXIII', 'IIIXII', 'IIIIXI', 'IIIIIX', 'YIIIII', 'IYIIII', 'IIYIII', 'IIIYII', 'IIIIYI', 'IIIIIY', 'ZIIIII', 'IZIIII', 'IIZIII', 'IIIZII', 'IIIIZI', 'IIIIIZ']


In [8]:
#generate q circuit
from pqk.Circuits import Circuits


qc =  Circuits.xyz_encoded(n_wire=6, full_ent=False)
qc.draw('text')

┌───────────┐┌───────────┐┌───────────┐
q_0: ┤ Rx(phi_0) ├┤ Ry(phi_0) ├┤ Rz(phi_0) ├
     ├───────────┤├───────────┤├───────────┤
q_1: ┤ Rx(phi_1) ├┤ Ry(phi_1) ├┤ Rz(phi_1) ├
     ├───────────┤├───────────┤├───────────┤
q_2: ┤ Rx(phi_2) ├┤ Ry(phi_2) ├┤ Rz(phi_2) ├
     ├───────────┤├───────────┤├───────────┤
q_3: ┤ Rx(phi_3) ├┤ Ry(phi_3) ├┤ Rz(phi_3) ├
     ├───────────┤├───────────┤├───────────┤
q_4: ┤ Rx(phi_4) ├┤ Ry(phi_4) ├┤ Rz(phi_4) ├
     ├───────────┤├───────────┤├───────────┤
q_5: ┤ Rx(phi_5) ├┤ Ry(phi_5) ├┤ Rz(phi_5) ├
     └───────────┘└───────────┘└───────────┘

In [113]:
from pqk.QEncoding import QEncoding

up_slide = 200


q_enc = QEncoding(data=X__np[:up_slide], obs=obs, qcircuit=qc)
en = q_enc.encode()

Encoding 200 data point
End data encoding


In [177]:
from pqk.QMeasures import QMeasures


testing_position = 8

# Assign parameters to the quantum circuit.
# 'inplace=False' means we get a new circuit with parameters bound.                
b_circuit = qc.assign_parameters(env.iloc[testing_position].to_numpy()[0:6], inplace=False)

mes =  QMeasures.StateVectorEstimator(b_circuit, obs)

print(env.iloc[testing_position])
print(mes)
print(en[testing_position])
print(y_np[testing_position])

illuminance   -0.981189
blinds         0.532931
lamps         -1.219187
rh            -0.766222
co2           -1.041931
temp          -0.061038
occupancy     -1.000000
Name: 8, dtype: float64
[-0.05705197  0.5256299   0.12082577  0.77002236  0.6350419   0.43385145
  0.06460041  0.81172947  0.84608639  0.62689353 -0.21527448  0.84627639
  0.99627901  0.25457508  0.5191714   0.11861734  0.74187512  0.30917501]
[-0.05705197  0.5256299   0.12082577  0.77002236  0.6350419   0.43385145
  0.06460041  0.81172947  0.84608639  0.62689353 -0.21527448  0.84627639
  0.99627901  0.25457508  0.5191714   0.11861734  0.74187512  0.30917501]
-1


In [181]:
#recreate data frame from encoding 
new_df = pd.DataFrame(en)
new_df['label'] = y_np[0:up_slide].tolist()
display(new_df)

new_df.to_csv('../qfm/fm/qencoding/xyz_18obsXYZ.csv', index=False)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,label
0,-0.057052,0.465385,0.204667,0.770022,0.635042,0.451721,0.064600,0.836198,0.865015,0.626894,-0.215274,0.840791,0.996279,0.290153,0.458105,0.118617,0.741875,0.298360,-1
1,-0.057052,0.300119,0.204667,0.770022,0.635042,0.451721,0.064600,0.868978,0.865015,0.626894,-0.215274,0.840791,0.996279,0.393455,0.458105,0.118617,0.741875,0.298360,-1
2,-0.057052,0.466032,0.204667,0.770022,0.635042,0.451721,0.064600,0.835972,0.865015,0.626894,-0.215274,0.840791,0.996279,0.289766,0.458105,0.118617,0.741875,0.298360,-1
3,-0.057052,0.262564,0.204667,0.770022,0.635042,0.451721,0.064600,0.869499,0.865015,0.626894,-0.215274,0.840791,0.996279,0.418369,0.458105,0.118617,0.741875,0.298360,-1
4,-0.057052,0.400041,0.204667,0.770022,0.635042,0.451721,0.064600,0.855062,0.865015,0.626894,-0.215274,0.840791,0.996279,0.329903,0.458105,0.118617,0.741875,0.298360,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.560369,-0.170711,0.173155,0.877681,0.635042,0.989929,-0.221051,0.536623,0.859723,-0.132369,-0.215274,-0.025030,0.798200,0.826373,0.480513,0.460602,0.741875,0.139336,1
196,0.560369,-0.140913,0.173155,0.877681,0.635042,0.989929,-0.221051,0.619269,0.859723,-0.132369,-0.215274,-0.025030,0.798200,0.772431,0.480513,0.460602,0.741875,0.139336,1
197,0.560369,-0.140913,0.173155,0.877681,0.635042,0.996088,-0.221051,0.619269,0.859723,-0.132369,-0.215274,-0.012637,0.798200,0.772431,0.480513,0.460602,0.741875,0.087462,1
198,0.560369,-0.140913,0.173155,0.877681,0.635042,0.991078,-0.221051,0.619269,0.859723,-0.132369,-0.215274,-0.022950,0.798200,0.772431,0.480513,0.460602,0.741875,0.131293,1


In [185]:
q_enc.save_encoding(file_name='prova',y_label= y_np[0:up_slide])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,label
0,-0.057052,0.465385,0.204667,0.770022,0.635042,0.451721,0.064600,0.836198,0.865015,0.626894,-0.215274,0.840791,0.996279,0.290153,0.458105,0.118617,0.741875,0.298360,-1
1,-0.057052,0.300119,0.204667,0.770022,0.635042,0.451721,0.064600,0.868978,0.865015,0.626894,-0.215274,0.840791,0.996279,0.393455,0.458105,0.118617,0.741875,0.298360,-1
2,-0.057052,0.466032,0.204667,0.770022,0.635042,0.451721,0.064600,0.835972,0.865015,0.626894,-0.215274,0.840791,0.996279,0.289766,0.458105,0.118617,0.741875,0.298360,-1
3,-0.057052,0.262564,0.204667,0.770022,0.635042,0.451721,0.064600,0.869499,0.865015,0.626894,-0.215274,0.840791,0.996279,0.418369,0.458105,0.118617,0.741875,0.298360,-1
4,-0.057052,0.400041,0.204667,0.770022,0.635042,0.451721,0.064600,0.855062,0.865015,0.626894,-0.215274,0.840791,0.996279,0.329903,0.458105,0.118617,0.741875,0.298360,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.560369,-0.170711,0.173155,0.877681,0.635042,0.989929,-0.221051,0.536623,0.859723,-0.132369,-0.215274,-0.025030,0.798200,0.826373,0.480513,0.460602,0.741875,0.139336,1
196,0.560369,-0.140913,0.173155,0.877681,0.635042,0.989929,-0.221051,0.619269,0.859723,-0.132369,-0.215274,-0.025030,0.798200,0.772431,0.480513,0.460602,0.741875,0.139336,1
197,0.560369,-0.140913,0.173155,0.877681,0.635042,0.996088,-0.221051,0.619269,0.859723,-0.132369,-0.215274,-0.012637,0.798200,0.772431,0.480513,0.460602,0.741875,0.087462,1
198,0.560369,-0.140913,0.173155,0.877681,0.635042,0.991078,-0.221051,0.619269,0.859723,-0.132369,-0.215274,-0.022950,0.798200,0.772431,0.480513,0.460602,0.741875,0.131293,1
